In [28]:
import pandas as pd
import os

In [29]:
path = 'data/Raw'
files = os.listdir(path)

In [30]:
estradas = pd.read_csv(f'{path}/dados_dos_quilometro_principal.csv', encoding='latin1', sep=';')

cat_cols = ['concessionaria', 'sentido', 'rodovia']
int_cols = ['ano_do_pnv_snv']

estradas[cat_cols] = estradas[cat_cols].astype('category')
estradas[int_cols] = estradas[int_cols].astype('int16')

estradas.to_parquet('data/dados_dos_quilometro_principal.parquet')


In [31]:
def correct_n_ocorrencia(row):
   try:
      return int(row.n_da_ocorrencia)
   except:
      split = row.n_da_ocorrencia.split('-')
      try:
         return int(split[0])
      except:
         split = row.n_da_ocorrencia.split('*')
         return int(split[0])
      
def correct_km(row):
   try:
      return float(row.km)
   except:
      split = row.km.split('+')
      try:
         return float(split[0])
      except:
         return (split[0][1:])
      
def process_and_append_csv(df, file):
   _ = pd.read_csv(f'{path}/{file}', encoding='latin1', sep=';')
   _['momento'] = pd.to_datetime(_.horario+' '+_.data, format = "%H:%M:%S %d/%m/%Y")
   _.drop(columns=['data', 'horario'], inplace=True)

   int_cols = ['n_da_ocorrencia', 'automovel', 'bicicleta',
       'caminhao', 'moto', 'onibus', 'outros', 'tracao_animal',
       'transporte_de_cargas_especiais', 'trator_maquinas', 'utilitarios',
       'ilesos', 'levemente_feridos', 'moderadamente_feridos',
       'gravemente_feridos', 'mortos']
    
   try:
      _[int_cols] = _[int_cols].astype('int16')
      _.km = _.km.astype('float64')
   except:
      _.n_da_ocorrencia = _.apply(correct_n_ocorrencia, axis = 1).astype('int32')
      _.km = _.apply(correct_km, axis = 1).astype('float64')
      _[int_cols] = _[int_cols].astype('int16')

   _.km = _.apply(lambda x: round(x.km), axis=1).astype('int32')
   return pd.concat([df, _], ignore_index=True)

In [32]:
df = pd.DataFrame({})
for file in files:
    if file.split('_')[0] == 'demostrativo':
        print(f'appending {file}')
        df = process_and_append_csv(df, file)

appending demostrativo_acidentes_ecoviasaraguaia.csv
appending demostrativo_acidentes_viacosteira.csv
appending demostrativo_acidentes_af.csv
appending demostrativo_acidentes_viabrasil.csv
appending demostrativo_acidentes_concer.csv
appending demostrativo_acidentes_eco101.csv


/tmp/ipykernel_283/312185802.py:24: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  _ = pd.read_csv(f'{path}/{file}', encoding='latin1', sep=';')


appending demostrativo_acidentes_arb.csv
appending demostrativo_acidentes_viabahia.csv
appending demostrativo_acidentes_cro.csv
appending demostrativo_acidentes_ecoponte.csv
appending demostrativo_acidentes_aps.csv
appending demostrativo_acidentes_ecoviasdocerrado.csv
appending demostrativo_acidentes_ecosul.csv
appending demostrativo_acidentes_als.csv


/tmp/ipykernel_283/312185802.py:24: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  _ = pd.read_csv(f'{path}/{file}', encoding='latin1', sep=';')


647.88
113.81
139.62
630.73
101.21
220.23
195.66
97.150
83.781
124.74
674.08
635.47
188.83
135.15
123.09
204.64
104.62
154.55
46.821
appending demostrativo_acidentes_ecoriominas.csv
appending demostrativo_acidentes_afd.csv
appending demostrativo_acidentes_concebra.csv
appending demostrativo_acidentes_epr_litoral_pioneiro.csv
appending demostrativo_acidentes_viaaraucaria.csv
appending demostrativo_acidentes_viasul.csv
appending demostrativo_acidentes_aco.csv
appending demostrativo_acidentes_trans.csv
appending demostrativo_acidentes_via040.csv
appending demostrativo_acidentes_msvia.csv
appending demostrativo_acidentes_eco050.csv


In [33]:
df.to_parquet('data/acidentes.parquet')